In [ ]:
"""
This file is part of lc-power-match-baluns.
Copyright © 2023 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook calculates component values for LC power matching baluns and displays circuit diagrams for the calculated networks.
"""

In [ ]:
# import modules
import lc_power_match_baluns.topology
import numpy as np

In [ ]:
# inputs

# design frequency (Hz)
f0 = 300e6

# impedance at balanced port (Ohm)
Zb = 73 + 43j

# impedance at unbalanced port (Ohm)
Zu = 50 + 0j

# The Q-factors below are used to provide performance estimates at the design frequency.
# These performance estimates are only rough and do not account for non-ideal effects other than the Q-factor.
# Capacitors are modelled as series RC circuits and inductors as parallel RL circuits.
# These estimates are not shown in the ideal case where both Q-factors are set to infinity.

# capacitor Q-factor at the design frequency
Qc=2000

# inductor Q-factor at the design frequency
Ql=50

# whether to display TikZ code for circuit diagrams
printTikZ = False

In [ ]:
# draw templates
for cls in lc_power_match_baluns.topology.PowerMatchingBalunTopology.__subclasses__():
    if cls.netlist is not None:
        template_circuit = cls.lcapy_circuit()
        print(cls.name)
        template_circuit.draw()
        if printTikZ:
            print(template_circuit.sch._tikz_draw())

In [ ]:
# output results
for cls_index, cls in enumerate(lc_power_match_baluns.topology.PowerMatchingBalunTopology.__subclasses__()):
    print(f"\n{cls.name}")
    topology_solutions = cls.calculate_elements_from_impedances(Zb, Zu)
    if len(topology_solutions) == 0:
        print("No solutions")
        continue
    for solution_index, solution_reactances in enumerate(topology_solutions):
        print(f"\nSolution {solution_index + 1}")
        components = lc_power_match_baluns.oneport.SimpleLosslessOnePort.from_reactances_at_frequency(solution_reactances, f0)
        if cls.netlist is not None:
            circuit_lc = cls.lcapy_lc_circuit(components)
            circuit_lc.draw()
        for reactance_index, topology_reactance in enumerate(solution_reactances):
            print(f"X{reactance_index + 1} = {topology_reactance} Ohm")
        for component in components:
            print(component)
        if printTikZ:
            print(circuit_lc.sch._tikz_draw())
        if (np.isfinite(Ql) or np.isfinite(Qc)):
            lossy_element_impedances = [component.calculate_impedance(f0, Ql if isinstance(component, lc_power_match_baluns.oneport.Inductor) else Qc) for component in components]
            cmrr = cls.calculate_cmrr(Zb, Zu, lossy_element_impedances)
            insertion_loss = cls.calculate_insertion_loss(Zb, Zu, lossy_element_impedances)
            balanced_return_loss, unbalanced_return_loss = cls.calculate_return_losses(Zb, Zu, lossy_element_impedances)
            print(f"\nPerformance estimates at {f0} Hz:")
            print(f"CMRR: {cmrr} dB")
            print(f"Insertion loss: {insertion_loss} dB")
            print(f"Balanced port return loss: {balanced_return_loss} dB")
            print(f"Unbalanced port return loss: {unbalanced_return_loss} dB")